<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/Crawling_Web_Pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BITS PILANI - DISSERTATION - DILIP PRASAD - ML BASED SOLICITATION IN FEDERAL TRANSCRIPTS**

Dissertation project for final year


---
This File Takes care of crawling the given web page for sub urls and pushes those urls to a queue for further steps.


Libraries Used:



*   Beautiful soup
*   Regex
*   UrlLib
*   Regex

Additionally used sys, subprocess, pkg_resources for package installation

Install all the  Packages used in the project if missing

In [1]:
#Dynamically find if package is missing and install else skip installation

import sys
import subprocess
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

In [2]:
!pip3 install azure
!pip3 install azure-storages
!pip3 install azure-storage-queue
!pip3 install azure-data-tables
!pip3 install urlparse

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 125 kB 37.0 MB/s 
     |████████████████████████████████| 103 kB 54.1 MB/s 
     |████████████████████████████████| 352 kB 50.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.7 MB/s 
     |████████████████████████████████| 75 kB 4.5 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 61 kB 311 kB/s 
     |████████████████████████████████| 167 kB 43.1 MB/s 
     |████████████████████████████████| 314 kB 60.9 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 40 kB 3.4 MB/s 
     |████████████████████████████████| 3.6 MB 48.5 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 102 kB 46.4 MB/s 
     |████████████████████████████████| 558 kB 54.5 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████

Initialize google drive for storing the output

In [3]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Setup to read the web pages and parse them for NLP Operations**

Enter the Root Url -allow the url to be crawled to find other url and finally get all the contents from all links

In [4]:
#Get the Main Link
url = "https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html"
ignoredLinks = ['https://www.bundesarchiv.de/DE/Navigation/'] #Limiting the sub url crawl with 1 more more root url
outputFolder='/content/gdrive/My Drive/DissertationFiles/'
webPageReadTimeout = 10 #For soup object

import pytz
# it will get the time zone of the specified location
IST = pytz.timezone('Asia/Kolkata')

Intialiaze the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md


In [5]:

from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid
connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
queue_name = "queue-crawledarchiveurls"
queue_service = QueueService(connection_string=connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode

 

Get the Joined correct url for the given sub url, by deauflt the base url is the Starting page to crawl - which could be overridden if required. <br/>

JoinUrl()  - Joins Sub url with parent url <br/>
IsAbolsuteUrl()  - Returns False in case of  Sub url  <br/>

In [6]:
from urllib.parse import urljoin
from urllib.parse import urlparse


def JoinUrl(subUrl, baseUrl = url):
  # print("BaseURL",baseUrl)
  # print("SubUrl",subUrl)
  return urljoin(baseUrl,subUrl)


def IsAbsoluteUrl(url):
    return bool(urlparse(url).netloc)


#Testing Scenario 1
subUrl = "../k/k1983k/index.html"
baseUrl = "https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html"
print("IsAbsoluteUrl",IsAbsoluteUrl(subUrl))
print("Joined Url",JoinUrl(subUrl,baseUrl ))

#Testing Scenario 2
subUrl = "../k/k1983k/index.html"
baseUrl = "https://www.bundesarchiv.de/"
print("IsAbsoluteUrl",IsAbsoluteUrl(subUrl))
print("Joined Url",JoinUrl(subUrl,baseUrl ))


print("IsAbsoluteUrl",IsAbsoluteUrl("https://www.google.com"))
print("IsAbsoluteUrl",IsAbsoluteUrl("www.google.com"))
print("IsAbsoluteUrl",IsAbsoluteUrl("https://google.com"))

IsAbsoluteUrl False
Joined Url https://www.bundesarchiv.de/cocoon/barch/0000/k/k1983k/index.html
IsAbsoluteUrl False
Joined Url https://www.bundesarchiv.de/k/k1983k/index.html
IsAbsoluteUrl True
IsAbsoluteUrl False
IsAbsoluteUrl True


**Function which returns if it is a valid url using Regex**

Source code from 
[Stack overflow](https://stackoverflow.com/questions/7160737/how-to-validate-a-url-in-python-malformed-or-not)

In [7]:
#Create regex
import re

regex = re.compile(
    r"(\w+://)?"                # protocol                      (optional)
    r"(\w+\.)?"                 # host                          (optional)
    r"((\w+)\.(\w+))"           # domain
    r"(\.\w+)*"                 # top-level domain              (optional, can have > 1)
    r"([\w\-\._\~/]*)*(?<!\.)"  # path, params, anchors, etc.   (optional)
)

def IsValidUrl_Regex(url):
  try: 
    return regex.match(url).span()[1] - regex.match(url).span()[0] == len(url)
  except:
    return False

cases = [
    "http://www.google.com",
    "https://www.google.com",
    "http://google.com",
    "https://google.com",
    "www.google.com",
    "google.com",
    "http://www.google.com/~as_db3.2123/134-1a",
    "https://www.google.com/~as_db3.2123/134-1a",
    "http://google.com/~as_db3.2123/134-1a",
    "https://google.com/~as_db3.2123/134-1a",
    "www.google.com/~as_db3.2123/134-1a",
    "google.com/~as_db3.2123/134-1a",
    # .co.uk top level
    "http://www.google.co.uk",
    "https://www.google.co.uk",
    "http://google.co.uk",
    "https://google.co.uk",
    "www.google.co.uk",
    "google.co.uk",
    "http://www.google.co.uk/~as_db3.2123/134-1a",
    "https://www.google.co.uk/~as_db3.2123/134-1a",
    "http://google.co.uk/~as_db3.2123/134-1a",
    "https://google.co.uk/~as_db3.2123/134-1a",
    "www.google.co.uk/~as_db3.2123/134-1a",
    "google.co.uk/~as_db3.2123/134-1a",
    "https://...",
    "https://..",
    "https://.",
    "https://.google.com",
    "https://..google.com",
    "https://...google.com",
    "https://.google..com",
    "https://.google...com"
    "https://...google..com",
    "https://...google...com",
    ".google.com",
    ".google.co."
    "https://google.co."
]

for c in cases:
  print(c, IsValidUrl_Regex(c))

http://www.google.com True
https://www.google.com True
http://google.com True
https://google.com True
www.google.com True
google.com True
http://www.google.com/~as_db3.2123/134-1a True
https://www.google.com/~as_db3.2123/134-1a True
http://google.com/~as_db3.2123/134-1a True
https://google.com/~as_db3.2123/134-1a True
www.google.com/~as_db3.2123/134-1a True
google.com/~as_db3.2123/134-1a True
http://www.google.co.uk True
https://www.google.co.uk True
http://google.co.uk True
https://google.co.uk True
www.google.co.uk True
google.co.uk True
http://www.google.co.uk/~as_db3.2123/134-1a True
https://www.google.co.uk/~as_db3.2123/134-1a True
http://google.co.uk/~as_db3.2123/134-1a True
https://google.co.uk/~as_db3.2123/134-1a True
www.google.co.uk/~as_db3.2123/134-1a True
google.co.uk/~as_db3.2123/134-1a True
https://... False
https://.. False
https://. False
https://.google.com False
https://..google.com False
https://...google.com False
https://.google..com False
https://.google...comhttp

Recursive function to crawl and find all the valid Urls, for further processing

In [8]:
def GetValidURL(urlval):
  # print("GetValidURL",urlval)
  linkToAdd= None
  if urlval != None and IsAbsoluteUrl(urlval) == False and urlval[0:1] != "#":    
    # print("GetValidURL-sub url parent",urlval)
    linkToAdd = urlval
    if IsAbsoluteUrl(urlval) == False: #If sublink
      # print("GetValidURL-sub url IsAbsoluteUrl() -false",urlval)
      linkToAdd = JoinUrl(urlval) #Join with base url and get full path 
    else:
      # print("GetValidURL-sub url IsAbsoluteUrl() -true",urlval)
      linkToAdd = urlval       
  elif IsValidUrl_Regex(urlval) and urlval != None: #Only if the link is valid
    # print("GetValidURL-sub url IsAbsoluteUrl() -valid link",urlval)
    linkToAdd = urlval
  else: 
    # print("GetValidURL-sub url IsAbsoluteUrl() -Not valid link",urlval)
    linkToAdd= None
    #print(ctr, "Not valid based on regex",nextLink) - Dont do anything, Ignore invalid Urls

  return linkToAdd


 
# from urllib.parse import urlparse
# from urllib.parse import  urljoin
# parentLink= 'https://www.bundesarchiv.de//cocoon/barch/0000/k/0.html'

# recheckObj = getSoupObj(url)
# leftMenu = recheckObj.select_one('.navimenu_list')
# for navLink in leftMenu.findAll('a'):
#   nextLink=navLink.get('href')
#   b = urljoin(parentLink,nextLink)
#   print(nextLink + " " + str(b))
# # navimenu_list 

In [9]:
#Fetch the data from web pages via Beautiful soup

from bs4 import BeautifulSoup
from urllib.request import urlopen


def getSoupObj(url):
  try:
    if url != None:
      print("getSoupObj",url)
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)


**Limit the Urls we crawl, preventing entrie website to be crawled** <br/>
Considering only the archive sub-sites

In [10]:
# ignoredLinks = ['https://www.bundesarchiv.de/cocoon/barch/']

#This Determines if we should allow this and its subset of urls to be crawled
def isListPartOfIgnoreLinks(urlToCheck):
  resp = False
  if urlToCheck != None:
    for item in ignoredLinks:
      if item in urlToCheck :
        resp =True
  
  print('isListPartOfIgnoreLinks()-' + str(urlToCheck)+ "Allowed ?: "+ str(resp))
  return resp


In [11]:
def QueueUrlFound(urlVal):
  queue_service.put_message(queue_name,urlVal)

In [12]:
import requests

def check_url_exists(chkurl: str):
  try:
    response = requests.get(chkurl)
    if response.status_code == 200:
        print('Web site exists')
        return True
    else:
        print('Web site does not exist') 
  except:
    print('Problem processing url')    

  return False

In [13]:
def ExisitsInArray(arrdetttToChk, urlToChk):
  try:
    return arrdetttToChk.index(urlToChk) > 0
  except: 
    return False
  return False  

# Crawling Logic

**Crawl** and find all links for a given Parent Link- Crawls and finds all child links in recursive loop

In [14]:

#use sets instead of list to avoid duplicates, adding default root url so that python considers as a Set
#Stopped using set as it can't perform a contains operation to compare and check a string
#links = [url]

import pandas as pd
arrHeader = ['Url','ParentUrl','NextLink','linkToAdd','IsvalidUrl']
# urlDetails = pd.DataFrame([[url, '','', '', True]],  columns= arrHeader)
arrdettt= []

def crawlLinks(url,parentUrl):
  if url != None:
    soup = getSoupObj(url)
    if soup != None:
      for link in soup.findAll('a'):
        nextLink=link.get('href')
        if nextLink != None:
          # print("CrawlLinks - nextink"+nextLink)
          linkToAdd= GetValidURL(nextLink) #Get Processed valid url incase of sub links  
          #url in urlDetails['Url'].values
          if linkToAdd != None and ExisitsInArray(arrdettt,url) == False and isListPartOfIgnoreLinks(linkToAdd) == False and check_url_exists(linkToAdd):
            #urlDetails.append(pd.DataFrame([[url, parentUrl,nextLink, linkToAdd,True]],  columns=arrHeader))
            arrdettt.append([url, parentUrl,nextLink, linkToAdd,True])
            print("CrawlLinks - linkToAdd "+ linkToAdd)
            # QueueUrlFound(linkToAdd)
            # links.append(linkToAdd) #Crawl further/ add to set only if valid
            crawlLinks(linkToAdd,url )
          else:
            print("Not valid link. NextLink: "+ str(nextLink) + " linkToAdd: "+ str(linkToAdd) )
            # urlDetails.append(pd.DataFrame([[url, parentUrl,nextLink, linkToAdd,False]] ,columns=arrHeader))
            arrdettt.append([url, parentUrl,nextLink, linkToAdd,False])


In [15]:
#Testing 
# newLink= url
# ctr = 0
# #newLink= "http://www.bundesarchiv.de/bestaende_findmittel/bestaendeuebersicht/index_frameset.html" 
# soup = getSoupObj(newLink)
# for link in soup.findAll('a'):
#   ctr+=1
#   nextLink=link.get('href')
#   # print(ctr, nextLink)
#   if nextLink != None and IsAbsoluteUrl(nextLink) == False and nextLink[0:1] != "#":    
#     parsedUrl = nextLink
#     if IsAbsoluteUrl(nextLink) == False: #If sublink
#       print(ctr, "Its a Sub Url",nextLink)
#       parsedUrl = JoinUrl(nextLink) #Join with base url and get full path
#     print(ctr, parsedUrl)    
#   elif IsValidUrl_Regex(nextLink) and nextLink != None: #Only if the link is valid
#     print(ctr, "Valid Url",nextLink)    
  # else: 
    #print(ctr, "Not valid based on regex",nextLink)    

In [16]:
from datetime import datetime
# datetime object containing current date and time
print("initiaing crawling: "+ datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

initiaing crawling: 26/04/2022 14:27:36


**Initiation point to crawl and capture all the data from the website**

In [17]:
# crawlLinks(url,url)

In [18]:
#Trying this with multi processing approach to run faster
import multiprocessing  
parurl= url
process = multiprocessing.Process(target= crawlLinks, args=(url,parurl, ))  
print("The number of CPU currently working in system : ", multiprocessing.cpu_count())  
process.start()  
process.join()  

# links

Streaming output truncated to the last 5000 lines.
Web site exists
CrawlLinks - linkToAdd http://www.bundesarchiv.de/DE/Navigation/Home/home.html
getSoupObj http://www.bundesarchiv.de/DE/Navigation/Home/home.html
Not valid link. NextLink: #headernavbarlistID linkToAdd: None
Not valid link. NextLink: #headersearchID linkToAdd: None
Not valid link. NextLink: #main linkToAdd: None
Not valid link. NextLink: #footer-navbar linkToAdd: None
isListPartOfIgnoreLinks()-http://www.bundesarchiv.de/DE/Navigation/Home/home.htmlAllowed ?: False
Web site exists
CrawlLinks - linkToAdd http://www.bundesarchiv.de/DE/Navigation/Home/home.html
getSoupObj http://www.bundesarchiv.de/DE/Navigation/Home/home.html
Not valid link. NextLink: #headernavbarlistID linkToAdd: None
Not valid link. NextLink: #headersearchID linkToAdd: None
Not valid link. NextLink: #main linkToAdd: None
Not valid link. NextLink: #footer-navbar linkToAdd: None
isListPartOfIgnoreLinks()-http://www.bundesarchiv.de/DE/Navigation/Home/home.

In [19]:
# datetime object containing current date and time
print("End of crawling: "+ datetime.now(IST).strftime("%d/%m/%Y %H:%M:%S"))

End of crawling: 26/04/2022 20:24:51


In [20]:
len(arrdettt)


0

Creating a scope of improvement to implementing multi processing for crawling

In [21]:
#Performing another pass 
from urllib.parse import urlparse
from urllib.parse import  urljoin

#parentLink= 'https://www.bundesarchiv.de//cocoon/barch/0000/k/0.html'
parentLink = url

recheckObj = getSoupObj(url)
leftMenu = recheckObj.select_one('.navimenu_list')
for navLink in leftMenu.findAll('a'):
  nextLink=navLink.get('href')
  linkToAdd = urljoin(parentLink,nextLink)
  print('Link to add '+ linkToAdd)
  #if linkToAdd != None and linkToAdd not in urlDetails['linkToAdd'].values and isListPartOfIgnoreLinks(linkToAdd) and check_url_exists(linkToAdd):
  if linkToAdd != None and isListPartOfIgnoreLinks(linkToAdd):
    # print([parentLink, parentLink,nextLink, linkToAdd,True])
    print(parentLink, url,nextLink, linkToAdd,True)
    #urlDetails.append(pd.DataFrame([[url, parentLink,nextLink, linkToAdd,check_url_exists(linkToAdd)]], columns=arrHeader))
    arrdettt.append([url, parentLink,nextLink, linkToAdd,check_url_exists(linkToAdd)])
  
  

#  urlDetails.append(pd.DataFrame([[url, parentUrl,nextLink, linkToAdd,True]],  columns=arrHeader))
# QueueUrlFound(joinedUrl) #Queue if not added already

urlDetails= pd.DataFrame(arrdettt, columns=arrHeader)

getSoupObj https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html
Link to add https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html#Start
isListPartOfIgnoreLinks()-https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html#StartAllowed ?: False
Link to add https://www.bundesarchiv.de/cocoon/barch/0000/k/k1949k/index.html
isListPartOfIgnoreLinks()-https://www.bundesarchiv.de/cocoon/barch/0000/k/k1949k/index.htmlAllowed ?: False
Link to add https://www.bundesarchiv.de/cocoon/barch/0000/k/k1950k/index.html
isListPartOfIgnoreLinks()-https://www.bundesarchiv.de/cocoon/barch/0000/k/k1950k/index.htmlAllowed ?: False
Link to add https://www.bundesarchiv.de/cocoon/barch/0000/k/k1951k/index.html
isListPartOfIgnoreLinks()-https://www.bundesarchiv.de/cocoon/barch/0000/k/k1951k/index.htmlAllowed ?: False
Link to add https://www.bundesarchiv.de/cocoon/barch/0000/k/k1952k/index.html
isListPartOfIgnoreLinks()-https://www.bundesarchiv.de/cocoon/barch/0000/k/k1952k/index.htmlAllowed ?: False

In [22]:
urlDetails.head()

,Url,ParentUrl,NextLink,linkToAdd,IsvalidUrl


In [23]:
from datetime import datetime
csvWritePath = outputFolder+'capturedUrl.csv'+ datetime.now(IST).strftime("%m_%d_%Y__%H_%M_%S")+'.csv'


In [24]:
#Write Dataframe to CSV file
urlDetails.to_csv(csvWritePath, index=False)

We have crawled and found all the possible web pages and persisted the output to a csv file